<a href="https://colab.research.google.com/github/gabriele16/abinitio-train/blob/main/colab/abinitio-train-workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Molecular Dynamics simulations with NequIP, Allegro and CP2K

<center>
<img src="https://github.com/mir-group/allegro/blob/main/logo.png?raw=true" width="30%">
<img src="https://github.com/mir-group/nequip/blob/main/logo.png?raw=true" width="30%">
<center/>

<img src="https://github.com/gabriele16/cp2k/blob/master/tools/logo/cp2k_logo_500.png?raw=true" width="30%">




### Open in colab and change the runtime to use the GPU if you have not done so already

### Install dependencies, these include pytorch, nequip, allegro and other common packages


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%capture
! rm -rf abinitio-train
! git clone  https://github.com/gabriele16/abinitio-train.git
! pip install -r requirements.txt
!git clone --depth 1 https://github.com/mir-group/allegro.git
!pip install allegro/

In [4]:
# fix colab imports
import site
site.main()

# set to allow anonymous WandB
import os
os.environ["WANDB_ANONYMOUS"] = "must"


### Check Pytorch and whether it works correctly on the GPU

In [5]:
import torch
print("*****************************")
print("torch version: ", torch.__version__)
print("*****************************")
print("cuda is available: ", torch.cuda.is_available())
print("*****************************")
print("cuda version:")
!nvcc --version
print("*****************************")
print("check which GPU is being used (important only if attempting compilation of CP2K with CUDA):")
print("CP2K is optimized to run on the GPU with the folloging architectures: K20X, K40, K80, P100, V100, Mi50, Mi100, Mi250")
!nvidia-smi
print("*****************************")
print("check path where cuda is installed and adjust EXPORT PATH in the cp2k installation if necessary:")
print("*****************************")
!which nvcc

*****************************
torch version:  1.12.1+cu102
*****************************
cuda is available:  True
*****************************
cuda version:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
*****************************
check which GPU is being used (important only if attempting compilation of CP2K with CUDA):
CP2K is optimized to run on the GPU with the folloging architectures: K20X, K40, K80, P100, V100, Mi50, Mi100, Mi250
Thu May 11 16:20:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         M

Clone the CP2K repository

In [ ]:
%%capture
!rm -rf cp2k
! git clone --recursive https://github.com/gabriele16/cp2k.git cp2k

## Download a precompiled cp2k binary and run a test. 


In [ ]:
%%capture
! gdown https://drive.google.com/uc?id=1hbFxjV657q5C_09gOE8ebxQFIl23b50u
! tar -xzvf cp2k_prebuilt_cuda.tar.gz 

In [ ]:
! cp -r cp2k_prebuilt_cuda/exe cp2k/.
! cp -r cp2k_prebuilt_cuda/arch/* cp2k/arch/. 
! mv cp2k/tools/toolchain cp2k/tools/toolchain_not_built
! cp -r cp2k_prebuilt_cuda/toolchain cp2k/tools/.

In [ ]:
%%capture
! wget https://download.pytorch.org/libtorch/cu118/libtorch-cxx11-abi-shared-with-deps-2.0.0%2Bcu118.zip
! unzip /content/libtorch-cxx11-abi-shared-with-deps-2.0.0+cu118.zip

In [ ]:
! chmod 777 cp2k/exe/local_cuda/cp2k.ssmp &&  source /content/cp2k/tools/toolchain/install/setup 
! cd cp2k/tests/Fist/regtest-allegro/  &&  /content/cp2k/exe/local_cuda/cp2k.ssmp -i Allegro_si_MD.inp 

 DBCSR| CPU Multiplication driver                                           BLAS (U)
 DBCSR| Multrec recursion limit                                              512 (U)
 DBCSR| Multiplication stack size                                          30000 (D)
 DBCSR| Maximum elements for images                                    UNLIMITED (U)
 DBCSR| Multiplicative factor virtual images                                   1 (U)
 DBCSR| Use multiplication densification                                       F (D)
 DBCSR| Multiplication size stacks                                             3 (U)
 DBCSR| Use memory pool for CPU allocation                                     F (U)
 DBCSR| OMP: Current number of threads                                         4
 DBCSR| OMP: Max number of threads                                             4
 DBCSR| ACC: Number of devices/node                                            1
 DBCSR| ACC: Number of stack-buffers per thread                              

### Import PyTorch, numpy etc.
### Set-up the directory containing the training set

In [6]:
import numpy as np
import pandas as pd
import os

os.environ["WANDB_ANONYMOUS"] = "must"
from ase.io import read, write

import warnings
import os
data_dir = "/content/abinitio-train/datasets"

np.random.seed(0)
if torch.cuda.is_available():
  torch.cuda.manual_seed(0)
else:
  torch.manual_seed(0)  

### Workflow:
* Train: using a data set, train the neural network
* Evaluate: evaluate the error on total energies and forces using unseen data
* Deploy: convert the Python-based model into a stand-alone potential file for fast execution
* Run: run Molecular Dynamics in CP2K 

### Train a model

Here, we will train an Allegro potential

In [ ]:
! cd abinitio-train/datasets/ && tar -xzvf wat_bil_gra_aimd.tar.gz
! cd abinitio-train/datasets/ && tar -xzvf wat_gra_bil_film.tar.gz
! cd abinitio-train/datasets/refined_water_gra && for i in *.tar.gz; do tar -xzvf $i; done

wat_bil_gra_aimd/
wat_bil_gra_aimd/wat_pos_frc.extxyz
wat_bil_gra_aimd/celldata.dat
wat_gra_bil_film/
wat_gra_bil_film/wat_pos_frc.extxyz
wat_gra_bil_film/last_conf.xyz
wat_gra_bil_film/celldata.dat
wat_bil_gra_25/
wat_bil_gra_25/wat_pos_frc.extxyz
wat_bil_gra_25/celldata.dat
wat_bil_gra_25/last_conf.xyz
wat_bil_gra_30/
wat_bil_gra_30/wat_pos_frc.extxyz
wat_bil_gra_30/celldata.dat
wat_bil_gra_30/last_conf.xyz
wat_bil_gra_35/
wat_bil_gra_35/wat_pos_frc.extxyz
wat_bil_gra_35/celldata.dat
wat_bil_gra_35/last_conf.xyz
wat_bil_gra_40/
wat_bil_gra_40/wat_pos_frc.extxyz
wat_bil_gra_40/celldata.dat
wat_bil_gra_40/last_conf.xyz
wat_bil_gra_45/
wat_bil_gra_45/wat_pos_frc.extxyz
wat_bil_gra_45/celldata.dat
wat_bil_gra_45/last_conf.xyz
wat_bil_gra_60/
wat_bil_gra_60/wat_pos_frc.extxyz
wat_bil_gra_60/celldata.dat
wat_bil_gra_60/last_conf.xyz
wat_bil_gra_BULK/
wat_bil_gra_BULK/wat_pos_frc.extxyz
wat_bil_gra_BULK/celldata.dat
wat_bil_gra_BULK/last_conf.xyz
wat_bil_gra_FILM/
wat_bil_gra_FILM/wat_pos_f

In [7]:
! cd abinitio-train/datasets/ && tar -xzvf wat_film_gra.tar.gz

wat_film_gra/./
wat_film_gra/./wat_pos_frc.extxyz
wat_film_gra/./celldata.dat
wat_film_gra/./last_conf.xyz


### Below we explicitly write the input to train the model with NequIP/Allegro

In [9]:
allegro_input = """
# general
root: results/water-gra-film
run_name: water-gra-film
seed: 42
dataset_seed: 42
append: true
# To use float64 with cp2k, need to implement it, for now use float32
default_dtype: float64

# -- network --
model_builders:
 - allegro.model.Allegro
 # the typical model builders from `nequip` can still be used:
 - PerSpeciesRescale
 - ForceOutput
 - RescaleEnergyEtc

# cutoffs
r_max: 5.0
avg_num_neighbors: auto

# radial basis
# Try to use a small cutoff and a large polynomial cutoff p
# use p=48 in the Li3PO4 case in the paper since the cutoff is quite small
# see https://github.com/mir-group/allegro/discussions/20
BesselBasis_trainable: true 
### Use normalize_basis: true, see https://github.com/mir-group/allegro/discussions/20
normalize_basis: true
PolynomialCutoff_p: 48   

# symmetry
l_max: 2
parity: o3_full   

# Allegro layers:
num_layers: 2
env_embed_multiplicity: 8
embed_initial_edge: true

two_body_latent_mlp_latent_dimensions: [32, 64, 128]
two_body_latent_mlp_nonlinearity: silu
two_body_latent_mlp_initialization: uniform

latent_mlp_latent_dimensions: [128]
latent_mlp_nonlinearity: silu
latent_mlp_initialization: uniform
latent_resnet: true

env_embed_mlp_latent_dimensions: []
env_embed_mlp_nonlinearity: null
env_embed_mlp_initialization: uniform

# - end allegro layers -

# Final MLP to go from Allegro latent space to edge energies:
edge_eng_mlp_latent_dimensions: [32]
edge_eng_mlp_nonlinearity: null
edge_eng_mlp_initialization: uniform

include_keys:
  - user_label
key_mapping:
  user_label: label0

# -- data --
dataset: ase                                                                   
#dataset_file_name: /content/abinitio-train/datasets/wat_gra_bil_film/wat_pos_frc.extxyz                     # path to data set file
#dataset_file_name: /content/abinitio-train/datasets/refined_water_gra/wat_bil_gra_FILM/wat_pos_frc.extxyz                     # path to data set file
dataset_file_name: /content/abinitio-train/datasets/wat_film_gra/wat_pos_frc.extxyz                     # path to data set file

ase_args:
  format: extxyz

# A mapping of chemical species to type indexes is necessary if the dataset is provided with atomic numbers instead of type indexes.
#chemical_symbol_to_type:
chemical_symbols:
  - C
  - H
  - O

# logging
wandb: false
#wandb_project: allegro-water-tutorial
verbose: info
log_batch_freq: 10

# training
n_train: 100
n_val: 10
batch_size: 1
max_epochs: 200
learning_rate: 0.002
train_val_split: random
shuffle: true
metrics_key: validation_loss

# use an exponential moving average of the weights
use_ema: true
ema_decay: 0.99
ema_use_num_updates: true

# loss function
loss_coeffs:
  forces: 1.
  total_energy:
    - 1.
    - PerAtomMSELoss

# optimizer
optimizer_name: Adam
optimizer_params:
  amsgrad: false
  betas: !!python/tuple
  - 0.9
  - 0.999
  eps: 1.0e-08
  weight_decay: 0.

metrics_components:
  - - forces                               # key 
    - mae                                  # "rmse" or "mae"
  - - forces
    - rmse
  - - total_energy
    - mae    
  - - total_energy
    - mae
    - PerAtom: True                        # if true, energy is normalized by the number of atoms

# lr scheduler, drop lr if no improvement for 50 epochs
lr_scheduler_name: ReduceLROnPlateau
lr_scheduler_patience: 25
lr_scheduler_factor: 0.5

early_stopping_lower_bounds:
  LR: 1.0e-5

early_stopping_patiences:
  validation_loss: 25
"""

with open("allegro/configs/allegro-water-gra.yaml", "w") as f:
    f.write(allegro_input)

In [10]:
nequip_input = """

# IMPORTANT: READ THIS

# This is a full yaml file with all nequip options.
# It is primarily intented to serve as documentation/reference for all options
# For a simpler yaml file containing all necessary features to get you started, we strongly recommend to start with configs/example.yaml

# Two folders will be used during the training: 'root'/process and 'root'/'run_name'
# run_name contains logfiles and saved models
# process contains processed data sets
# if 'root'/'run_name' exists, 'root'/'run_name'_'year'-'month'-'day'-'hour'-'min'-'s' will be used instead.
root: results/water-gra-film
run_name: water-gra-film
seed: 123 # model seed
dataset_seed: 456 # data set seed
append: true # set true if a restarted run should append to the previous log file
default_dtype: float64 # type of float to use, e.g. float32 and float64
allow_tf32: false # whether to use TensorFloat32 if it is available
# device:  cuda                                                                   # which device to use. Default: automatically detected cuda or "cpu"

# network
r_max: 5.0 # cutoff radius in length units, here Angstrom, this is an important hyperparamter to scan
num_layers: 3 # number of interaction blocks, we find 3-5 to work best

l_max: 1 # the maximum irrep order (rotation order) for the network's features, l=1 is a good default, l=2 is more accurate but slower
parity: true # whether to include features with odd mirror parityy; often turning parity off gives equally good results but faster networks, so do consider this
num_features: 32 # the multiplicity of the features, 32 is a good default for accurate network, if you want to be more accurate, go larger, if you want to be faster, go lower

# alternatively, the irreps of the features in various parts of the network can be specified directly:
# the following options use e3nn irreps notation
# either these four options, or the above three options, should be provided--- they cannot be mixed.
# chemical_embedding_irreps_out: 32x0e                                              # irreps for the chemical embedding of species
# feature_irreps_hidden: 32x0o + 32x0e + 32x1o + 32x1e                              # irreps used for hidden features, here we go up to lmax=1, with even and odd parities; for more accurate but slower networks, use l=2 or higher, smaller number of features is faster
# irreps_edge_sh: 0e + 1o                                                           # irreps of the spherical harmonics used for edges. If a single integer, indicates the full SH up to L_max=that_integer
# conv_to_output_hidden_irreps_out: 16x0e                                           # irreps used in hidden layer of output block

nonlinearity_type: gate # may be 'gate' or 'norm', 'gate' is recommended
resnet:
  false # set true to make interaction block a resnet-style update
  # the resnet update will only be applied when the input and output irreps of the layer are the same

# scalar nonlinearities to use — available options are silu, ssp (shifted softplus), tanh, and abs.
# Different nonlinearities are specified for e (even) and o (odd) parity;
# note that only tanh and abs are correct for o (odd parity).
# silu typically works best for even
nonlinearity_scalars:
  e: silu
  o: tanh

nonlinearity_gates:
  e: silu
  o: tanh

# radial network basis
num_basis: 8 # number of basis functions used in the radial basis, 8 usually works best
BesselBasis_trainable: true # set true to train the bessel weights
PolynomialCutoff_p: 48 # p-exponent used in polynomial cutoff function, smaller p corresponds to stronger decay with distance

# radial network
invariant_layers: 2 # number of radial layers, usually 1-3 works best, smaller is faster
invariant_neurons: 64 # number of hidden neurons in radial function, smaller is faster
avg_num_neighbors: auto # number of neighbors to divide by, null => no normalization, auto computes it based on dataset
use_sc: true # use self-connection or not, usually gives big improvement

# to specify different parameters for each convolutional layer, try examples below
# layer1_use_sc: true                                                             # use "layer{i}_" prefix to specify parameters for only one of the layer,
# priority for different definitions:
#   invariant_neurons < InteractionBlock_invariant_neurons < layer{i}_invariant_neurons

# data set
# there are two options to specify a dataset, npz or ase
# npz works with npz files, ase can ready any format that ase.io.read can read
# in most cases working with the ase option and an extxyz file is by far the simplest way to do it and we strongly recommend using this
# simply provide a single extxyz file that contains the structures together with energies and forces (generated with ase.io.write(atoms, format='extxyz', append=True))

include_keys:
  - user_label
key_mapping:
  user_label: label0

# alternatively, you can read directly from a VASP OUTCAR file (this will only read that single OUTCAR)
# # for VASP OUTCAR, the yaml input should be
# dataset: ase
# dataset_file_name: OUTCAR
# ase_args:
#   format: vasp-out
# important VASP note: the ase vasp parser stores the potential energy to "free_energy" instead of "energy".
# Here, the key_mapping maps the external name (key) to the NequIP default name (value)
# key_mapping:
#   free_energy: total_energy

# npz example
# the keys used need to be stated at least once in key_mapping, npz_fixed_field_keys or include_keys
# key_mapping is used to map the key in the npz file to the NequIP default values (see data/_key.py)
# all arrays are expected to have the shape of (nframe, natom, ?) except the fixed fields
# note that if your data set uses pbc, you need to also pass an array that maps to the nequip "pbc" key
# dataset: npz                                                                       # type of data set, can be npz or ase
# dataset_url: http://quantum-machine.org/gdml/data/npz/toluene_ccsd_t.zip           # url to download the npz. optional
# dataset_file_name: ./benchmark_data/toluene_ccsd_t-train.npz                       # path to data set file
# key_mapping:
#   z: atomic_numbers                                                                # atomic species, integers
#   E: total_energy                                                                  # total potential eneriges to train to
#   F: forces                                                                        # atomic forces to train to
#   R: pos                                                                           # raw atomic positions
# npz_fixed_field_keys:                                                              # fields that are repeated across different examples
#   - atomic_numbers

# A list of chemical species found in the data. The NequIP atom types will be named after the chemical symbols and ordered by atomic number in ascending order.
# (In this case, NequIP's internal atom type 0 will be named H and type 1 will be named C.)
# Atoms in the input will be assigned NequIP atom types according to their atomic numbers.
chemical_symbols:
  - C
  - H
  - O

# Alternatively, you may explicitly specify which chemical species in the input will map to NequIP atom type 0, which to atom type 1, and so on.
# Other than providing an explicit order for the NequIP atom types, this option behaves the same as `chemical_symbols`
#chemical_symbol_to_type:
#  C: 0
#  H: 1
#  O: 2

# Alternatively, if the dataset has type indices, you may give the names for the types in order:
# (this also sets the number of types)
# type_names:
#   - my_type
#   - atom
#   - thing

# As an alternative option to npz, you can also pass data ase ASE Atoms-objects
# This can often be easier to work with, simply make sure the ASE Atoms object
# has a calculator for which atoms.get_potential_energy() and atoms.get_forces() are defined
dataset: ase
#dataset_file_name: ./abinitio-train/datasets/wat_gra_bil_film/wat_pos_frc.extxyz # need to be a format accepted by ase.io.read
#dataset_file_name: /content/abinitio-train/datasets/refined_water_gra/wat_bil_gra_FILM/wat_pos_frc.extxyz                     # path to data set file
dataset_file_name: /content/abinitio-train/datasets/wat_film_gra/wat_pos_frc.extxyz                     # path to data set file

ase_args: # any arguments needed by ase.io.read
  format: extxyz

# If you want to use a different dataset for validation, you can specify
# the same types of options using a `validation_` prefix:
# validation_dataset: ase
# validation_dataset_file_name: xxx.xyz                                            # need to be a format accepted by ase.io.read

# logging
#wandb: true # we recommend using wandb for logging
#wandb_project: water-example # project name used in wandb
#wandb_watch: false

# see https://docs.wandb.ai/ref/python/watch
# wandb_watch_kwargs:
#   log: all
#   log_freq: 1
#   log_graph: true

verbose: info # the same as python logging, e.g. warning, info, debug, error. case insensitive
log_batch_freq: 1 # batch frequency, how often to print training errors withinin the same epoch
log_epoch_freq: 1 # epoch frequency, how often to print
save_checkpoint_freq: -1 # frequency to save the intermediate checkpoint. no saving of intermediate checkpoints when the value is not positive.
save_ema_checkpoint_freq: -1 # frequency to save the intermediate ema checkpoint. no saving of intermediate checkpoints when the value is not positive.

# training
n_train: 100 # number of training data
n_val: 10 # number of validation data
learning_rate: 0.005 # learning rate, we found values between 0.01 and 0.005 to work best - this is often one of the most important hyperparameters to tune
batch_size: 1 # batch size, we found it important to keep this small for most applications including forces (1-5); for energy-only training, higher batch sizes work better
validation_batch_size: 1 # batch size for evaluating the model during validation. This does not affect the training results, but using the highest value possible (<=n_val) without running out of memory will speed up your training.
max_epochs: 200 # stop training after _ number of epochs, we set a very large number here, it won't take this long in practice and we will use early stopping instead
train_val_split: random # can be random or sequential. if sequential, first n_train elements are training, next n_val are val, else random, usually random is the right choice
shuffle: true # If true, the data loader will shuffle the data, usually a good idea
metrics_key: validation_loss # metrics used for scheduling and saving best model. Options: `set`_`quantity`, set can be either "train" or "validation, "quantity" can be loss or anything that appears in the validation batch step header, such as f_mae, f_rmse, e_mae, e_rmse
use_ema: true # if true, use exponential moving average on weights for val/test, usually helps a lot with training, in particular for energy errors
ema_decay: 0.99 # ema weight, typically set to 0.99 or 0.999
ema_use_num_updates: true # whether to use number of updates when computing averages
report_init_validation: true # if True, report the validation error for just initialized model

# early stopping based on metrics values.
# LR, wall and any keys printed in the log file can be used.
# The key can start with Training or validation. If not defined, the validation value will be used.
early_stopping_patiences: # stop early if a metric value stopped decreasing for n epochs
  validation_loss: 50

early_stopping_delta: # If delta is defined, a decrease smaller than delta will not be considered as a decrease
  validation_loss: 0.005

early_stopping_cumulative_delta: false # If True, the minimum value recorded will not be updated when the decrease is smaller than delta

early_stopping_lower_bounds: # stop early if a metric value is lower than the bound
  LR: 1.0e-5

early_stopping_upper_bounds: # stop early if a metric value is higher than the bound
  cumulative_wall: 1.0e+100

# loss function
loss_coeffs: # different weights to use in a weighted loss functions
  forces: 1 # if using PerAtomMSELoss, a default weight of 1:1 on each should work well
  total_energy:
    - 1
    - PerAtomMSELoss

# # default loss function is MSELoss, the name has to be exactly the same as those in torch.nn.
# the only supprted targets are forces and total_energy

# here are some example of more ways to declare different types of loss functions, depending on your application:
# loss_coeffs:
#   total_energy: MSELoss
#
# loss_coeffs:
#   total_energy:
#   - 3.0
#   - MSELoss
#
# loss_coeffs:
#   total_energy:
#   - 1.0
#   - PerAtomMSELoss
#
# loss_coeffs:
#   forces:
#   - 1.0
#   - PerSpeciesL1Loss
#
# loss_coeffs: total_energy
#
# loss_coeffs:
#   total_energy:
#   - 3.0
#   - L1Loss
#   forces: 1.0

# output metrics
metrics_components:
  - - forces # key
    - mae # "rmse" or "mae"
  - - forces
    - rmse
  - - forces
    - mae
    - PerSpecies: True # if true, per species contribution is counted separately
      report_per_component: False # if true, statistics on each component (i.e. fx, fy, fz) will be counted separately
  - - forces
    - rmse
    - PerSpecies: True
      report_per_component: False
  - - total_energy
    - mae
  - - total_energy
    - mae
    - PerAtom: True # if true, energy is normalized by the number of atoms

# optimizer, may be any optimizer defined in torch.optim
# the name `optimizer_name`is case sensitive
optimizer_name: Adam # default optimizer is Adam
optimizer_amsgrad: false
optimizer_betas: !!python/tuple
  - 0.9
  - 0.999
optimizer_eps: 1.0e-08
optimizer_weight_decay: 0

# gradient clipping using torch.nn.utils.clip_grad_norm_
# see https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html#torch.nn.utils.clip_grad_norm_
# setting to inf or null disables it
max_gradient_norm: null

# lr scheduler, currently only supports the two options listed below, if you need more please file an issue
# first: on-plateau, reduce lr by factory of lr_scheduler_factor if metrics_key hasn't improved for lr_scheduler_patience epoch
lr_scheduler_name: ReduceLROnPlateau
lr_scheduler_patience: 100
lr_scheduler_factor: 0.5

# second, cosine annealing with warm restart
# lr_scheduler_name: CosineAnnealingWarmRestarts
# lr_scheduler_T_0: 10000
# lr_scheduler_T_mult: 2
# lr_scheduler_eta_min: 0
# lr_scheduler_last_epoch: -1

# we provide a series of options to shift and scale the data
# these are for advanced use and usually the defaults work very well
# the default is to scale the energies and forces by scaling them by the force standard deviation and to shift the energy by its mean
# in certain cases, it can be useful to have a trainable shift/scale and to also have species-dependent shifts/scales for each atom

per_species_rescale_scales_trainable: false
# whether the scales are trainable. Defaults to False. Optional
per_species_rescale_shifts_trainable: false
# whether the shifts are trainable. Defaults to False. Optional
per_species_rescale_shifts: dataset_per_atom_total_energy_mean
# initial atomic energy shift for each species. default to the mean of per atom energy. Optional
# the value can be a constant float value, an array for each species, or a string
# string option include:
# *  "dataset_per_atom_total_energy_mean", which computes the per atom average
# *  "dataset_per_species_total_energy_mean", which automatically compute the per atom energy mean using a GP model
per_species_rescale_scales: dataset_forces_rms
# initial atomic energy scale for each species. Optional.
# the value can be a constant float value, an array for each species, or a string
# string option include:
# *  "dataset_per_atom_total_energy_std", which computes the per atom energy std
# *  "dataset_per_species_total_energy_std", which uses the GP model uncertainty
# *  "dataset_per_species_forces_rms", which compute the force rms for each species
# If not provided, defaults to dataset_per_species_force_rms or dataset_per_atom_total_energy_std, depending on whether forces are being trained.
# per_species_rescale_kwargs:
#   total_energy:
#     alpha: 0.1
#     max_iteration: 20
#     stride: 100
# keywords for GP decomposition of per specie energy. Optional. Defaults to 0.1
# per_species_rescale_arguments_in_dataset_units: True
# if explicit numbers are given for the shifts/scales, this parameter must specify whether the given numbers are unitless shifts/scales or are in the units of the dataset. If ``True``, any global rescalings will correctly be applied to the per-species values.

# global energy shift and scale
# When "dataset_total_energy_mean", the mean energy of the dataset. When None, disables the global shift. When a number, used directly.
# Warning: if this value is not None, the model is no longer size extensive
global_rescale_shift: null

# global energy scale. When "dataset_force_rms", the RMS of force components in the dataset. When "dataset_total_energy_std", the stdev of energies in the dataset. When null, disables the global scale. When a number, used directly.
# If not provided, defaults to either dataset_force_rms or dataset_total_energy_std, depending on whether forces are being trained.
global_rescale_scale: dataset_forces_rms

# whether the shift of the final global energy rescaling should be trainable
global_rescale_shift_trainable: false

# whether the scale of the final global energy rescaling should be trainable
global_rescale_scale_trainable: false
# # full block needed for per specie rescale
# global_rescale_shift: null
# global_rescale_shift_trainable: false
# global_rescale_scale: dataset_forces_rms
# global_rescale_scale_trainable: false
# per_species_rescale_trainable: true
# per_species_rescale_shifts: dataset_per_atom_total_energy_mean
# per_species_rescale_scales: dataset_per_atom_total_energy_std

# # full block needed for global rescale
# global_rescale_shift: dataset_total_energy_mean
# global_rescale_shift_trainable: false
# global_rescale_scale: dataset_forces_rms
# global_rescale_scale_trainable: false
# per_species_rescale_trainable: false
# per_species_rescale_shifts: null
# per_species_rescale_scales: null

# Options for e3nn's set_optimization_defaults. A dict:
# e3nn_optimization_defaults:
#   explicit_backward: True
"""

!mkdir nequip_train
with open("nequip_train/water-gra.yaml", "w") as f:
    f.write(nequip_input)

### Train with Allegro or NequIP

In [11]:
!rm -rf ./results

Model='NequIP'

if Model=='Allegro':
  !nequip-train allegro/configs/allegro-water-gra.yaml  
elif Model=='NequIP':
  !nequip-train nequip_train/water-gra.yaml --equivariance-test
else:
  print("Model has to be either Allegro or NequIP")


Streaming output truncated to the last 5000 lines.
     55    71      0.00338      0.00338     5.97e-06       0.0384         0.05       0.0358       0.0282       0.0496       0.0379       0.0453       0.0363        0.064       0.0485        0.756       0.0021
     55    72      0.00366      0.00365     3.67e-06       0.0405        0.052       0.0368       0.0335       0.0521       0.0408       0.0463       0.0431       0.0657       0.0517        0.593      0.00165
     55    73      0.00281      0.00277     3.74e-05       0.0355       0.0453       0.0332       0.0351       0.0403       0.0362       0.0414       0.0446       0.0526       0.0462         1.89      0.00526
     55    74      0.00297      0.00297      6.5e-06       0.0363       0.0468       0.0331       0.0302       0.0465       0.0366       0.0425       0.0372       0.0588       0.0461        0.789      0.00219
     55    75      0.00271      0.00265      5.8e-05       0.0347       0.0443       0.0332       0.0274       0.

In [12]:
!nequip-train --help

usage: nequip-train
       [-h]
       [--equivariance-test [EQUIVARIANCE_TEST]]
       [--model-debug-mode]
       [--grad-anomaly-mode]
       [--log LOG]
       config

Train (or
restart
training
of) a
NequIP
model.

positional arguments:
  config
    YAML file
    configuring
    the model,
    dataset,
    and other
    options

options:
  -h, --help
    show this
    help
    message and
    exit
  --equivariance-test [EQUIVARIANCE_TEST]
    test the
    model's equ
    ivariance
    before
    training on
    n (default
    1) random
    frames from
    the dataset
  --model-debug-mode
    enable
    model debug
    mode, which
    can
    sometimes
    give much
    more useful
    error
    messages at
    the cost of
    some speed.
    Do not use
    for
    production
    training!
  --grad-anomaly-mode
    enable
    PyTorch
    autograd
    anomaly
    mode to
    debug NaN
    gradients.
    Do not use
    for
    production
    training!
  --log LOG
    log file to
    

## Evaluate the test error
##### We get rather small errors in the forces of ~50 meV/A

In [13]:
! nequip-evaluate --train-dir results/water-gra-film/water-gra-film --batch-size 1

Using device: cuda
Loading model... 
loaded model from training session
Loading original dataset...
Loaded dataset specified in config.yaml.
Using origial training dataset (5793 frames) minus training (100 frames) and validation frames (10 frames), yielding a test set size of 5683 frames.
Starting...
  0% 0/5683 [00:00<?, ?it/s]

  0% 1/5683 [00:00<1:12:35,  1.30it/s]
  0% 2/5683 [00:01<1:03:15,  1.50it/s]
  0% 3/5683 [00:02<1:25:33,  1.11it/s]
  0% 4/5683 [00:02<59:23,  1.59it/s]  
  0% 5/5683 [00:02<44:58,  2.10it/s]
  0% 6/5683 [00:03<36:14,  2.61it/s]
  0% 7/5683 [00:03<30:39,  3.08it/s]
  0% 8/5683 [00:03<27:02,  3.50it/s]
  0% 9/5683 [00:03<24:38,  3.84it/s]
  0% 10/5683 [00:03<22:59,  4.11it/s]
  0% 11/5683 [00:04<21:49,  4.33it/s]
  0% 12/5683 [00:04<20:59,  4.50it/s]
  0% 13/5683 [00:04<20:23,  4.63it/s]
  0% 14/5683 [00:04<19:59,  4.73it/s]
  0% 15/5683 [00:04<19:43,  4.79it/s]
  0% 16/5683 [00:05<19:33,  4.83it/s]
  0% 17/5683 [00:05<19:26,  4.86it/s]
  0% 18/5683 [00:05<19:

### Deploy the model

We now convert the model to a potential file. This makes it independent of NequIP and we can load it in CP2K to run MD.

In [14]:
from datetime import datetime
import subprocess

# datetime object containing current date and time
depl_time = datetime.now().strftime("%d%m%Y-%H%M")

if Model == "Allegro":
  !nequip-deploy build --train-dir results/water-gra-film/water-gra-film water-gra-film-deploy-alle.pth
  cmd = ["cp", "water-gra-film-deploy-alle.pth", "water-gra-film-deploy-alle-"+depl_time+".pth"] 
elif Model == "NequIP":
   !nequip-deploy build --train-dir results/water-gra-film/water-gra-film water-gra-film-deploy-neq.pth 
   cmd = ["cp", "water-gra-film-deploy-neq.pth", "water-gra-film-deploy-neq-"+depl_time+".pth"] 

subprocess.Popen(cmd)

INFO:root:Loading best_model from training session...
INFO:root:Compiled & optimized model.


<Popen: returncode: None args: ['cp', 'water-gra-film-deploy-neq.pth', 'wate...>

In [15]:
model_is_pretrained = False

if model_is_pretrained == False:
  ! cp *2023*.pth /content/drive/MyDrive/models_and_datasets/models/.
else:
  ! cp /content/drive/MyDrive/models_and_datasets/models/*.pth .

# CP2K

We are now in a position to run MD with our potential.

Set up a simple cp2k input file, you can find an example file inside ```cp2k/tests/Fist/regtest-allegro```




CAUTION: Be careful with the units of the .pth model file. One can explicitly specify the units of this file since CP2K input defaults as Angstrom for lengths and A.U. for everything else.

### We now run MD of bulk water at 360 K using 64 molecules in the NVT ensemble for 10 ps (20,000 steps)

---



In [ ]:
from ase.build import sort
from ase.io import read, write

last_conf = sort(read('abinitio-train/datasets/refined_water_gra/wat_bil_gra_FILM/wat_pos_frc.extxyz',index=-1))
write('last_conf.xyz',last_conf)


In [ ]:
cp2k_input_md = """
&GLOBAL
  PROJECT water_gra
  RUN_TYPE MD
&END GLOBAL
&FORCE_EVAL
  METHOD FIST
  &MM
    &FORCEFIELD
     &NONBONDED
     &ALLEGRO
        ATOMS C H O
        PARM_FILE_NAME ./water-gra-film-deploy-alle.pth
        UNIT_COORDS angstrom
        UNIT_ENERGY Hartree
        UNIT_FORCES Hartree*Bohr^-1
     &END ALLEGRO
    &END NONBONDED
    &END FORCEFIELD
    &POISSON
      &EWALD
        EWALD_TYPE none
      &END EWALD
    &END POISSON
  &END MM
  &SUBSYS
    &CELL
       ABC 24.68 25.648 48.0
#      MULTIPLE_UNIT_CELL 2 2 2
    &END CELL
    &TOPOLOGY
#     MULTIPLE_UNIT_CELL 2 2 2
      COORD_FILE_NAME ./last_conf.xyz
      COORD_FILE_FORMAT XYZ
    &END TOPOLOGY
  &END SUBSYS
&END FORCE_EVAL
&MOTION
#  &CONSTRAINT
#    &FIXED_ATOMS
#      LIST 1..240
#    &END
#  &END
  &MD
    ENSEMBLE NVT
    STEPS 20000
    TIMESTEP 0.5
    TEMPERATURE 300
    &THERMOSTAT
      &CSVR
        TIMECON 10
      &END CSVR
    &END
  &END MD
&END MOTION
"""

!mkdir cp2k_run
with open("cp2k_run/Allegro_md.inp", "w") as f:
    f.write(cp2k_input_md)

mkdir: cannot create directory ‘cp2k_run’: File exists


In [ ]:
cp2k_input_md = """
&GLOBAL
  PROJECT water_gra
  RUN_TYPE MD
&END GLOBAL
&FORCE_EVAL
  METHOD FIST
  &MM
    &FORCEFIELD
     &NONBONDED
     &NequIP
        ATOMS C H O
        PARM_FILE_NAME ./water-gra-film-deploy-neq.pth
        UNIT_COORDS angstrom
        UNIT_ENERGY Hartree
        UNIT_FORCES Hartree*Bohr^-1
     &END NequIP
    &END NONBONDED
    &END FORCEFIELD
    &POISSON
      &EWALD
        EWALD_TYPE none
      &END EWALD
    &END POISSON
  &END MM
  &SUBSYS
    &CELL
       ABC 24.68 25.648 48.0
#      MULTIPLE_UNIT_CELL 2 2 2
    &END CELL
    &TOPOLOGY
#     MULTIPLE_UNIT_CELL 2 2 2
      COORD_FILE_NAME ./last_conf.xyz
      COORD_FILE_FORMAT XYZ
    &END TOPOLOGY
  &END SUBSYS
&END FORCE_EVAL
&MOTION
#  &CONSTRAINT
#    &FIXED_ATOMS
#      LIST 1
#    &END
#  &END
  &MD
    ENSEMBLE NVT
    STEPS 20000
    TIMESTEP 0.5
    TEMPERATURE 300
    &THERMOSTAT
      &CSVR
        TIMECON 10
      &END CSVR
    &END
  &END MD
&END MOTION
"""

!mkdir cp2k_run
with open("cp2k_run/NequIP_md.inp", "w") as f:
    f.write(cp2k_input_md)

mkdir: cannot create directory ‘cp2k_run’: File exists


## Run 20,000 MD steps


In [ ]:
! cp *.pth last_conf.xyz cp2k_run/.
if Model == "Allegro":
  ! cd /content/cp2k_run && ../cp2k/exe/local_cuda/cp2k.ssmp -i Allegro_md.inp 
elif Model == "NequIP":
  ! cd /content/cp2k_run && ../cp2k/exe/local_cuda/cp2k.ssmp -i NequIP_md.inp   

Streaming output truncated to the last 5000 lines.
 MD_VEL| Centre of mass motion (COM)
 MD_VEL| VCOM [a.u.]              0.0000000000    -0.0000000000     0.0000000000

 MD| ***************************************************************************
 MD| Step number                                                              74
 MD| Time [fs]                                                         37.000000
 MD| Conserved quantity [hartree]                            -0.108411365536E+05
 MD| ---------------------------------------------------------------------------
 MD|                                          Instantaneous             Averages
 MD| CPU time per MD step [s]                      0.221039             0.486120
 MD| Energy drift per atom [K]           0.135637063994E+04   0.938324511867E+03
 MD| Potential energy [hartree]         -0.108611016645E+05  -0.108582897797E+05
 MD| Kinetic energy [hartree]            0.198667887374E+02   0.146233702911E+02
 MD| Temperature [K]

### Visualize the trajectory with ase and ngl

In [ ]:
from ase.visualize import view

In [ ]:
wat_traj = read("/content/cp2k_run/water_gra-pos-1.xyz", index="::")

In [ ]:
! head /content/cp2k_run/*.ener

#     Step Nr.          Time[fs]        Kin.[a.u.]          Temp[K]            Pot.[a.u.]        Cons Qty[a.u.]        UsedTime[s]
         0            0.000000         2.691951383       300.000000000    -10851.947239399    -10849.254812994         0.000000000
         1            0.500000         2.851866779       317.821502782    -10852.033473492    -10849.181094859         2.816104081
         2            1.000000         3.370181535       375.584220117    -10852.311859131    -10848.941124282        12.563596373
         3            1.500000         4.130002769       460.261221139    -10852.718474865    -10848.587868300         0.225819904
         4            2.000000         4.996967540       556.878653842    -10853.180751324    -10848.183117308         3.874344316
         5            2.500000         5.866185866       653.747230073    -10853.642271519    -10847.775342240         0.241787414
         6            3.000000         6.689877013       745.542106334    -10854.07

In [ ]:
#water_cell_prod = np.array([9.85, 9.85, 9.85])
water_cell_prod = np.array([24.68000, 25.64800,48.00000 ])

for i in range(len(wat_traj)):
    wat_traj[i].cell = water_cell_prod
    wat_traj[i].pbc = np.array([True, True, True])
    wat_traj[i].wrap()

In [ ]:
%%capture 
! pip install nglview
!jupyter-nbextension enable nglview --py --sys-prefix
import nglview as nv

In [ ]:
from google.colab import output

output.enable_custom_widget_manager()

In [ ]:
view(wat_traj, viewer="ngl")

### Calculate radial distribution function with MDAnalysis

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import rdf

ModuleNotFoundError: ignored

In [ ]:
reader = mda.coordinates.XYZ.XYZReader("/content/cp2k_run/water-pos-1.xyz")
topology = mda.topology.XYZParser.XYZParser("/content/cp2k_run/water-pos-1.xyz")

u = mda.Universe("/content/cp2k_run/water-pos-1.xyz")

u.dimensions = [water_cell_prod[0], water_cell_prod[1], water_cell_prod[2], 90.0, 90.0, 90.0]

In [ ]:
O_at = u.select_atoms("name O")
H_at = u.select_atoms("name H")

Ordf = rdf.InterRDF(
    O_at,
    O_at,
    nbins=100,  # default
    range=(0.00001, water_cell_prod[0]/2),  # distance in angstroms
)
Ordf.run(start = 10000)

OHrdf = rdf.InterRDF(
    O_at,
    H_at,
    nbins=50,  # default
    range=(0.00001, water_cell_prod[0]/2),  # distance in angstroms
)
OHrdf.run(start = 10000)

HHrdf = rdf.InterRDF(
    H_at,
    H_at,
    nbins=50,  # default
    range=(0.00001, water_cell_prod[0]/2),  # distance in angstroms
)
HHrdf.run(start = 10000)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(Ordf.bins, Ordf.rdf)
plt.xlabel("Radius (angstrom)")
plt.ylabel("Radial distribution")

### You can compare the O-O rdf above with the one obtained from a longer AIMD trajectory with the SCAN functional. The two rdfs are rather close to each other.

https://www.pnas.org/doi/suppl/10.1073/pnas.2121641119/suppl_file/pnas.2121641119.sapp.pdf

In [ ]:
plt.plot(OHrdf.bins, OHrdf.rdf)
plt.xlabel("Radius (angstrom)")
plt.ylabel("Radial distribution")

In [ ]:
plt.plot(HHrdf.bins, HHrdf.rdf)
plt.xlabel("Radius (angstrom)")
plt.ylabel("Radial distribution")

## Plot Total Energy and Temperature as a function of time after 5ps of equilibration to check the overall stability of the MD

In [ ]:
import pandas as pd
ener_file = pd.read_csv("/content/cp2k_run/water-1.ener", delim_whitespace=True, header = 0)

ener_file = ener_file.drop(columns="#").rename(columns = {"Step" : "Time[fs]", "Nr.": "Kin.[a.u.]",
                                              "Time[fs]": "Temp[K]",
                                              "Kin.[a.u.]": "Pot [a.u.]","Temp[K]": "Cons Qty[a.u.]",
                                              "Pot.[a.u.]": "Used Time[s]"
                                              }).drop(columns=["Cons","Qty[a.u.]","UsedTime[s]"])
ener_file[10000:].plot(x="Time[fs]",y="Pot [a.u.]")
ener_file[10000:].plot(x="Time[fs]",y="Temp[K]")
